In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
image_generator = ImageDataGenerator(
    rescale=1./255.,
    shear_range=0.1,
    zoom_range=0.3,
)
image_generator_test = ImageDataGenerator(
    rescale=1./255.
)

In [3]:
train_data = image_generator.flow_from_directory(
    'data_xray',
    target_size=(500, 500),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=10
)

Found 550 images belonging to 2 classes.


In [4]:
test_data = image_generator_test.flow_from_directory(
    'test_data_xray',
    target_size=(500, 500),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=10
)

Found 74 images belonging to 2 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_data.classes), y=train_data.classes)

In [8]:
class_weights = dict(zip(np.unique(train_data.classes), weights))

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
rlrop = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=0.001)

In [10]:
mch = ModelCheckpoint('chestxray.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=30, validation_data=test_data, validation_steps=len(test_data), callbacks=[es, rlrop, mch], class_weight=class_weights)

Epoch 1/30
55/55 [==============================] - 2494s 45s/step - loss: 0.9598 - accuracy: 0.6564 - val_loss: 0.6494 - val_accuracy: 0.5405 - lr: 0.0010
Epoch 2/30
55/55 [==============================] - 1938s 35s/step - loss: 0.6166 - accuracy: 0.7291 - val_loss: 0.4657 - val_accuracy: 0.8378 - lr: 0.0010
Epoch 3/30
